In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np
import bz2
import re

In [2]:
train = "/content/drive/MyDrive/irfinalproject/train.ft.txt.bz2"
# test = "/content/drive/MyDrive/test.ft.txt.bz2"
users = "/content/drive/MyDrive/irfinalproject/Reviews.csv"

In [6]:
d=pd.read_csv(users)
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [10]:
import random
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
      
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
  
    return np.array(labels), texts
labels, texts = get_labels_and_texts(train)

In [11]:
df=pd.DataFrame()
df['Label'] = labels
df['Text'] = texts
df['ProductId'] = d['ProductId']
df['UserId'] = d['UserId']
df['Username'] = d['ProfileName']
df['HelpfulnessNumerator']=d['HelpfulnessNumerator']
df['HelpfulnessDenominator']=d['HelpfulnessDenominator']

In [12]:
df = df[:d.shape[0]]
df = df.sample(n=36000) 

In [13]:
import random
df['Score'] = np.zeros(36000)
for i in range(len(df)):
  if(df['Label'].iloc[i]==1):
    df['Score'].iloc[i]=random.randint(4,5)
  else:
    df['Score'].iloc[i]=random.randint(1,3)

<ipython-input-13-48f9b9e7b6ca>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score'].iloc[i]=random.randint(4,5)
<ipython-input-13-48f9b9e7b6ca>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score'].iloc[i]=random.randint(1,3)


In [14]:
d=d.drop('Id',axis=1)
d=d.drop('Time',axis=1)
d=d.drop('Summary',axis=1)

d.rename(columns = {'ProfileName':'Username'}, inplace = True)

d['Label'] = d['Score'].apply(lambda x : 1 if x>3 else 0)

In [15]:
d = d.sample(n=14000)

In [16]:
df=df.append(d)

<ipython-input-16-ec0b14f945d8>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(d)


In [17]:
df.shape

(50000, 8)

In [18]:
df

,Label,Text,ProductId,UserId,Username,HelpfulnessNumerator,HelpfulnessDenominator,Score
504174,1,The Creatures deliver the goods.: As a long ti...,B003P02EGU,A3LAN99BT1QBEN,Mrs. H.,1.0,1.0,4.0
533066,0,Don't buy used: Don't buy this used. Neither o...,B009E7YC54,A16F8IW5NK9Y4Q,Kristin Ames,1.0,1.0,3.0
264589,0,Belts burning the casing: Loved this vacuum---...,B002ZOPVG2,AIQDUKFB4X38M,VonMalcolm,1.0,1.0,1.0
290222,1,"Queensryche, has out-done its self with this c...",B000IZ0OC6,A1N1PMGMVFD8FD,gotjava,0.0,1.0,4.0
99874,1,Lexar multi card reader: The unit combines all...,B000H1CO74,A1GYEGLX3P2Y7P,Shala Kerrigan,1.0,1.0,5.0
...,...,...,...,...,...,...,...,...
168722,1,"Great product, excellent value. I was buying t...",B005PANYEY,A3FABTXTZJ2ZN7,Fryday17,0.0,0.0,5.0
64747,0,I was extremely dissapointed in this product. ...,B003WEDT4U,A1NNJKJV1V04JA,Marka,0.0,4.0,1.0
239457,1,I am head over heels about these peanuts. They...,B001FA1IZA,A3BADG55FYI5M2,Gary M.,0.0,0.0,5.0
161228,0,I have a 60lb English Bulldog/German Shepherd ...,B008O3G2K2,A3DH53FPYSP9E,Media Man,2.0,4.0,2.0


In [19]:
df.to_csv("file.csv")